# **Pengertian Crawling**

Crawling adalah proses yang digunakan oleh search engine untuk menjelajahi atau merayapi website untuk mendapatkan informasi sebagai respons terhadap kueri yang diberikan. Proses ini dilakukan dengan bantuan bot yang dikenal dengan nama web crawler atau crawler. Crawler adalah mesin pencari bot yang khusus digunakan untuk mencari, mengunjungi, dan menganalisis konten situs web dan informasi lainnya di internet. Crawling data bertujuan untuk mengumpulkan data secara otomatis dari website dan membangun indeks data untuk digunakan mesin pencari.

## Proses Crawling

In [6]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

### Tools Atau Library



*   Pandas digunakan untuk membuat dataframe agar mudah dibaca.
*   Selenium digunakan untuk mengotomatiskan browser agar dapat berinteraksi dengan elemen-elemen di halaman web.
*   Time berfungsi untuk memberikan jeda ketika ingin berpindah halaman.





In [7]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

## Proses Coding

### Menginisialisasi browser Chrome secara otomatis dengan opsi headless, tanpa tampilan grafis.

In [8]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

### Panggil fungsi web_driver(), dan arahkan otomatis ke link tujuan dengan sellenium, dan baut list kosong untuk menampung hasil crawling.

In [9]:
driver = web_driver()


kategori_urls = ["https://mojok.co/liputan/kuliner/", "https://mojok.co/liputan/kampus/"]
data_list = []

### Mengunjungi setiap url berita, mengambil informasi judul, isi, kategori, tanggal, lalu menyimpannya.

In [10]:
for main_url in kategori_urls:
    count = 0  # Variabel untuk menghitung jumlah artikel yang diambil per kategori
    driver.get(main_url)

    while True:
        try:
            # Ambil semua elemen <h3> yang ada di dalam <div class="jeg_inner_content">
            contoh = driver.find_elements(By.XPATH, '//div[@class="jeg_inner_content"]//h3')

            # Ambil semua elemen <a> di dalam setiap <h3>
            urls = [a.get_attribute('href') for elem in contoh for a in elem.find_elements(By.TAG_NAME, "a")]

            # Loop melalui setiap URL artikel
            for url in urls:
                if count >= 5:  # Batasi hingga 10 data per kategori
                    break

                try:
                    driver.get(url)
                    time.sleep(2)

                    # Ekstrak data yang dibutuhkan
                    judul = driver.find_element(By.XPATH, './/h1[@class="jeg_post_title"]').text
                    kategori = driver.find_element(By.XPATH, './/span[@class="breadcrumb_last_link"]/a').text
                    tanggal = driver.find_element(By.XPATH, './/div[@class="jeg_meta_date"]/a').text

                    isi = driver.find_element(By.XPATH, './/div[@class="content-inner "]')
                    isibos = [p.text for p in isi.find_elements(By.TAG_NAME, "p")]

                    data = {
                        "Judul": judul,
                        "Kategori": kategori,
                        "Tanggal": tanggal,
                        "Isi": " ".join(isibos)
                    }
                    data_list.append(data)

                    count += 1  # Tambah penghitung setelah data diambil

                except Exception as e:
                    print(f"Data not found on {url}. Error: {e}")

                # Kembali ke halaman utama setelah mengambil data dari artikel
                driver.back()
                time.sleep(2)

            if count >= 5:  # Hentikan jika sudah mencapai batas 10 per kategori
                break

            # Mencari tombol "Next" untuk pagination
            try:
                next_button = driver.find_element(By.XPATH, '//a[contains(@class, "page_nav next")]')
                next_button.click()
                time.sleep(3)
            except:
                print("No more pages found or 'Next' button not available.")
                break

        except Exception as e:
            print(f"An error occurred during pagination: {e}")
            break


### Menutup browser dan mengakhiri sesi Selenium

In [11]:
driver.quit()

### Convert Dataframe

In [12]:
df = pd.DataFrame(data_list)
df

,Judul,Kategori,Tanggal,Isi
0,Kerja di Pasar Minggu Jakarta Selatan Bikin Ta...,Kuliner,4 September 2024,"Kerja di Pasar Minggu, Jakarta Selatan, sebena..."
1,Warung Makan Barokah Sih Surabaya: Rp8 Ribu Su...,Kuliner,23 Agustus 2024,Di antara semakin banyaknya warung makan yang ...
2,Mengulik “Rahasia” Warung Soto Sleman hingga k...,Kuliner,15 Agustus 2024,Menemukan warung soto seharga Rp2 ribu di Slem...
3,Kedai Kopi TEKO.SU Sleman: Dari Vespa dan Stig...,Kuliner,13 Agustus 2024,"TEKO.SU, sebuah street coffee di Sleman, Jogja..."
4,"Kupi Harvest, Warung Makan Nyempil Penolong Pe...",Kuliner,8 Agustus 2024,Mahasiswa UGM punya banyak tempat makan andala...
5,"September Hitam di FISIP UNAIR, Usaha Pembangu...",Kampus,5 September 2024,Teman-teman aktivis dan mahasiswa sedang menit...
6,Tips Khusus Lulus Jurusan Kedokteran Tepat Wak...,Kampus,3 September 2024,"Di balik prestisiusnya kuliah Kedokteran, ada ..."
7,Cerita Mahasiswa Jogja “Kabur” dari Sidang Skr...,Kampus,3 September 2024,"Padahal tinggal sidang skripsi, tapi Adhi (23)..."
8,Ospek di Universitas BSI: Mahasiswa Baru Disam...,Kampus,30 Agustus 2024,Semakin ke sini antusiasme calon mahasiswa bar...
9,"4 Panduan sebelum Gabung Organisasi Kampus, Bi...",Kampus,21 Agustus 2024,Mahasiswa baru sering kali bimbang perihal iku...
